In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torchsampler import ImbalancedDatasetSampler
import matplotlib.pyplot as plt
import os
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from functools import reduce
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix

In [2]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [6]:
train_dir = "C:/Users/33783/Desktop/start_deep/start_deep/Projet_ML/train_images"
test_dir = "C:/Users/33783/Desktop/start_deep/start_deep/Projet_ML/test_images"

transform = transforms.Compose(
    [transforms.Grayscale(), 
     transforms.ToTensor(), 
     transforms.Normalize(mean=(0,),std=(1,))])


train_data = torchvision.datasets.ImageFolder(train_dir, transform=transform)
test_data = torchvision.datasets.ImageFolder(test_dir, transform=transform)

valid_size = 0.2
batch_size = 32

#Split train and validation

num_train = len(train_data)
indices_train = list(range(num_train))
np.random.shuffle(indices_train)
split_tv = int(np.floor(valid_size * num_train))
train_new_idx, valid_idx = indices_train[split_tv:],indices_train[:split_tv]

train_sampler = ImbalancedDatasetSampler(train_data,indices=train_new_idx)
valid_sampler = ImbalancedDatasetSampler(train_data,indices=valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=1)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=1)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=1)
classes = ('noface','face')

print()
print('Noface in train data', train_loader.dataset.targets.count(0))
print('Face in train data',train_loader.dataset.targets.count(1))
print()
print('Noface in valid data', valid_loader.dataset.targets.count(0))
print('Face in valid data',valid_loader.dataset.targets.count(1))
print()
print('Noface in test data',test_loader.dataset.targets.count(0))
print('Face in test data',test_loader.dataset.targets.count(1))
print()



Noface in train data 26950
Face in train data 64770

Noface in valid data 26950
Face in valid data 64770

Noface in test data 6831
Face in test data 632



In [131]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = reduce(lambda r,s: r*s,size)
        #print(num_features)
        return num_features
    
#Analytics
def validate(valid_loader,show=True) : 

    for i, data in enumerate(valid_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            outputs = net(inputs)
            y_pred_softmax = torch.log_softmax(outputs, dim = 1)
            _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
            all_predicted.extend(np.array(y_pred_tags))
            all_labels.extend(np.array(labels))
    
    f1s=f1_score(all_labels,all_predicted,average='weighted')
    prec=precision_score(all_labels,all_predicted,average='weighted')
    rec=recall_score(all_labels,all_predicted,average='weighted')
    cf_m=confusion_matrix(all_labels,all_predicted,normalize='true')
    
    if show ==True : 
        print('Validation set')
        print("f1_score",f1s)
        print("precision",prec)
        print("recall",rec)
        print("confusion matrix\n", cf_m)
        
    return f1s,prec,rec,cf_m

In [132]:
#pour le gpu
"""
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

net.to(device)
inputs, labels = data[0].to(device), data[1].to(device)
"""

'\ndevice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")\nprint(device)\n\nnet.to(device)\ninputs, labels = data[0].to(device), data[1].to(device)\n'

In [133]:
#Parameters definition
net = Net()
print(net)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
all_labels=[]
all_predicted=[]
all_losses=[]
all_accuracies=[]
all_f1scores=[]
epochs=25
#Training
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for n_batch,batch in enumerate(train_loader) : 
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = batch
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        writer.add_scalar("Loss/train", loss, epoch) #visualization

        loss.backward()
        optimizer.step()

        all_labels.extend(np.array(labels))
            
        _, predicted = torch.max(outputs.data, 1)
        all_predicted.extend(np.array(predicted))

        running_loss += loss.item()
        if n_batch % 2000 == 1999:    # print every 10 mini-batches
            
            #peut-être ça ne sert plus---------
            y_pred_softmax = torch.log_softmax(outputs, dim = 1)
            _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
            outputs=np.array(y_pred_tags)
            labels=np.array(labels)
            #---------------------------------
            all_losses.append(running_loss / n_batch)
            f1 = validate(valid_loader,False)[0]
            all_f1scores.append(f1)
            print('[%2d, %2d] loss: %.3f f1_score on validation set : %.3f' %
                  (epoch + 1, n_batch + 1, running_loss / n_batch,f1))
            
            running_loss = 0.0
            
plt.plot(all_losses, color='blue')
plt.plot(all_f1scores, color='red')

print('Finished Training')

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
[ 1, 2000] loss: 0.546 f1_score on validation set : 0.783
[ 2, 2000] loss: 0.157 f1_score on validation set : 0.867


KeyboardInterrupt: 

In [ ]:
validate(valid_loader,True)


In [ ]:
validate(test_loader,True)

In [ ]:
saved_model_path='C:/Users/33783/Desktop/start_deep/start_deep/firstModel'
torch.save(net.state_dict(), saved_model_path )

In [ ]:
net = Net()
net.load_state_dict(torch.load(saved_model_path))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(2):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

In [ ]:

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


 #get some random training images
from collections import Counter
dataiter = iter(train_loader) #list of 5 batches
for batch in dataiter : 
    images, labels = batch #dataiter.next()
    l = list(map(lambda x:x.item(),labels))
    c = Counter(l)
    print(c[0]/c[1])
    # show images
    imshow(torchvision.utils.make_grid(images))
    #print(' '.join('%5s' % classes[labels[j]] for j in range(len(images))))
